In [1]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [32]:
from IPython.display import display

yeast_type = aq.SampleType.find_by_name('Yeast Strain')

def mating(a, b, marker, description):
    y1 = aq.Sample.find(a)
    y2 = aq.Sample.find(b)
    assert y1
    assert y2
    
    haploids = sorted([y1, y2], key=lambda x: x.name)
    
    return aq.Sample.new(
        name = description,
        sample_type_id=yeast_type.id,
        description=description,
        project='SD2 2021 Redesigns',
        properties={
            'Haploids': haploids,
            'Mating Type': 'Diploid',
            'Has this strain passed QC?': 'Yes',
            'Integrated Marker(s)': marker
        }
    )

import pandas as pd

x = ',Description,Haploid 1,Haploid 2,Marker\n0,DSGRN NOR (BAD),36489,36501,"TRP, LEU"\n1,DSGRN NOR (GOOD),36536,36537,"TRP, LEU"\n2,DSGRN OR (BAD),36489,36497,"TRP, BLEO"\n3,DSGRN OR (GOOD),36498,36499,"TRP, BLEO"\n4,GANDER NOR (BAD),36399,36381,"TRP, URA"\n5,GANDER NOR (GOOD),36400,36382,"TRP, URA"\n6,GANDER OR (BAD),36500,36538,"TRP, LEU"\n7,GANDER OR (GOOD),36491,36539,"TRP, LEU"\n8,GANDER OR (BAD) (v2),36500,36540,"TRP, LEU"\n'

with open('mating.csv', 'w') as f:
    f.write(x)
    

df = pd.read_csv('mating.csv', index_col=0)
display(df)

samples = []
for _, row in df.iterrows():
    s = mating(row['Haploid 1'], row['Haploid 2'], marker=row['Marker'], description=row['Description'])
    samples.append(s)
    
for s in samples:
    if aq.Sample.find_by_name(s.name):
        tokens = s.name.split()
        topo = tokens[0]
        circuit = tokens[1]
        cdm = tokens[2]
        if len(tokens) == 4:
            version = ' ' + tokens[3]
        else:
            version = ''
            
        if 'BAD' in cdm:
            perf = 'low performance'
            x='low'
        else:
            perf = 'high performance'
            x='high'
        name = 'Dual Inducible {circuit} circuit ({topo} topology, {perf} CDM)'.format(
            circuit=circuit,
            topo=topo,
            perf=perf
        )
        description = 'Doxycycline and Beta-estradiol inducible {circuit} circuit. {topo} topology. Predicted {perf} gRNA selection by CDM.'.format(
            circuit=circuit,
            topo=topo,
            perf=perf
        )
        s.name = name + version
        s.description = description
        s.save()


,Description,Haploid 1,Haploid 2,Marker
0,DSGRN NOR (BAD),36489,36501,"TRP, LEU"
1,DSGRN NOR (GOOD),36536,36537,"TRP, LEU"
2,DSGRN OR (BAD),36489,36497,"TRP, BLEO"
3,DSGRN OR (GOOD),36498,36499,"TRP, BLEO"
4,GANDER NOR (BAD),36399,36381,"TRP, URA"
5,GANDER NOR (GOOD),36400,36382,"TRP, URA"
6,GANDER OR (BAD),36500,36538,"TRP, LEU"
7,GANDER OR (GOOD),36491,36539,"TRP, LEU"
8,GANDER OR (BAD) (v2),36500,36540,"TRP, LEU"


TridentRequestError: Error response:
Name The sample name 'Dual Inducible NOR circuit (DSGRN topology, low performance CDM)' is the name of an existing sample

In [36]:
s = [36564,
36565,
36566,
36567,
36568,
36569,
36570,
36571]

for sid in s:
    print('UWBF_' + str(aq.Sample.find(sid).id))

UWBF_36564
UWBF_36565
UWBF_36566
UWBF_36567
UWBF_36568
UWBF_36569
UWBF_36570
UWBF_36571


In [25]:
Doxycycline and Beta-estradiol inducible OR circuit. Gander topology. Predicted high performance gRNA selection by CDM.

In [44]:
aq.Sample.find(36566).properties['Haploids'][0].name

'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | HIS-HIS<7XTetO-pMinCyc1-URGR-W8> | URA-URA<pGRR-W10W8-URGR-W5> (v2)'

In [39]:
aq.Sample.find(36109).name

'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-GPD-UAS-25-pCYC1-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pREV1-rtTA-VPR | pMOD8-7TetO-pMinCYC-RGR-r2'

In [45]:
s = [
    36108,
36109,
36110,
36111,
36112,
36113,
36114,
36115,
36116,
36117]

for _s in s:
    print(aq.Sample.find(_s).name)

CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-yeGFP(broken)
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-GPD-UAS-25-pCYC1-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-RGR-r2
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-pGRR-r9-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-URGR-r9
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-pGRR-r7-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-URGR-r7
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-pGRR-r10-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-URGR-r10
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-pGRR-r5-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL18B-rtTA-VP16 | pMOD8-7TetO-pMinCYC-URGR-r5
CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD6-pGRR-r1-yeGFP _X_ CEN.PK2 - MAT alpha ::: pMOD-HO-KanMX-pRPL